# Ensemble Learning

## Initial Imports

In [27]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [29]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split

## Read the CSV and Perform Basic Data Cleaning

In [30]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head(5)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [31]:
# Create features
X = df.drop(['loan_status', 'home_ownership', 'verification_status', 'issue_d', 'pymnt_plan', 'initial_list_status', 'next_pymnt_d', 'application_type', 
             'hardship_flag', 'debt_settlement_flag'], axis=1)

# Create target
y = df['loan_status']

In [32]:
# Check the balance of target values
Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

In [33]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,
                                                   y,
                                                   random_state=1)

In [34]:
df.columns

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [40]:
# Create the StandardScaler instance
from sklearn.preprocessing import LabelEncoder, StandardScaler
scaler = StandardScaler()

# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
# Binary encoding using Pandas (single column)
binary_encoded = pd.get_dummies(df, columns=["loan_status"])
binary_encoded.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,loan_status_high_risk,loan_status_low_risk
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,n,27.24,0.0,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,0,1
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,n,20.23,0.0,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,0,1
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,n,24.26,0.0,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,0,1
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,n,31.44,0.0,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,0,1
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,n,18.76,0.0,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,0,1


In [42]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [43]:
# Scale the training and testing data
StandardScaler(copy=True, with_mean=True, with_std=True)

StandardScaler()

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [44]:
# Resample the training data with the BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [46]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_brf = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_brf)

0.7023150163472849

In [47]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_brf)

array([[   60,    41],
       [ 3240, 13864]], dtype=int64)

In [48]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.59      0.81      0.04      0.69      0.47       101
   low_risk       1.00      0.81      0.59      0.89      0.69      0.49     17104

avg / total       0.99      0.81      0.60      0.89      0.69      0.49     17205



In [51]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sorted[:11]

[(0.0655635817623315, 'total_rec_int'),
 (0.06528036081873223, 'last_pymnt_amnt'),
 (0.06277129875620881, 'total_pymnt_inv'),
 (0.05985163433565138, 'total_rec_prncp'),
 (0.055053063169639246, 'total_pymnt'),
 (0.033776751297507714, 'int_rate'),
 (0.019291167552657758, 'out_prncp_inv'),
 (0.018948515278747545, 'dti'),
 (0.018793237773830143, 'mths_since_recent_inq'),
 (0.017864791938440518, 'tot_hi_cred_lim'),
 (0.01672966122818627, 'tot_cur_bal')]

### Easy Ensemble Classifier

In [52]:
# Train the Classifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=78)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=78)

In [54]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_eec_pred)

0.7318831099667498

In [55]:
confusion_matrix(y_test, y_eec_pred)

array([[   70,    31],
       [ 3922, 13182]], dtype=int64)

In [56]:
# Print the imbalanced classification report
balanced_accuracy_score(y_test, y_eec_pred)

0.7318831099667498

### Final Questions

1. Which model had the best balanced accuracy score?

    Random Forest. 

2. Which model had the best recall score?

    RF. 

3. Which model had the best geometric mean score?

    RF

4. What are the top three features?

    RF.